In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 96
compression = 0.4
dropout_rate = 0.2

In [0]:
#!pip install PyDrive
#import os
#from pydrive.drive import GoogleDrive
#from pydrive.auth import GoogleAuth
#gauth = GoogleAuth()
#drive = GoogleDrive(gauth)
#file_list = drive.ListFile(
#    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()
#(x_train, y_train), (x_test, y_test) = drive.GetList()
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

#from google.colab import files
#uploaded = files.upload()

In [21]:
'''
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())
  '''

'\n!pip install -U -q PyDrive\nimport os\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n# 1. Authenticate and create the PyDrive client.\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\n# choose a local (colab) directory to store the data.\nlocal_download_path = os.path.expanduser(\'~/data\')\ntry:\n  os.makedirs(local_download_path)\nexcept: pass\n\n# 2. Auto-iterate using the query syntax\n#    https://developers.google.com/drive/v2/web/search-parameters\nfile_list = drive.ListFile(\n    {\'q\': "\'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk\' in parents"}).GetList()\n\nfor f in file_list:\n  # 3. Create & download by id.\n  print(\'title: %s, id: %s\' % (f[\'title\'], f[\'id\']))\n  fname = os.path.join(local_download_path, f[\'title\'])\n  print(\'downloading to {}\'.format(fname))\n  f

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
'''

datagen = ImageDataGenerator(rotation_range=90)
datagen.fit(x_train)
'''
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(x_train)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 96, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 96, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [28]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 64)   1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 64)   256         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 64)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [29]:
'''

filepath='dataFile.h5'
checkpoint=ModelCheckpoint(filepath,monitor='val_acc',verbose=1, save_best_only=True,mode='max')
callback=[checkpoint]
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) // batch_size, epochs=epochs,
                   callbacks=callback,
                   validation_data=(x_test, y_test),
                   validation_steps=x_test.shape[0] // batch_size)
'''

"\n\nfilepath='dataFile.h5'\ncheckpoint=ModelCheckpoint(filepath,monitor='val_acc',verbose=1, save_best_only=True,mode='max')\ncallback=[checkpoint]\nmodel.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\n                    steps_per_epoch=len(x_train) // batch_size, epochs=epochs,\n                   callbacks=callback,\n                   validation_data=(x_test, y_test),\n                   validation_steps=x_test.shape[0] // batch_size)\n"

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [31]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
43776/50000 [=========================>....] - ETA: 58s - loss: 1.4692 - acc: 0.4635

50000/50000 [==============================] - 504s 10ms/step - loss: 1.4302 - acc: 0.4782 - val_loss: 1.2471 - val_acc: 0.5628
Epoch 2/50
26112/50000 [==============>...............] - ETA: 3:38 - loss: 1.0063 - acc: 0.6401

50000/50000 [==============================] - 490s 10ms/step - loss: 0.9602 - acc: 0.6566 - val_loss: 1.0481 - val_acc: 0.6462
Epoch 3/50
19392/50000 [==========>...................] - ETA: 4:40 - loss: 0.8045 - acc: 0.7144

50000/50000 [==============================] - 490s 10ms/step - loss: 0.7771 - acc: 0.7247 - val_loss: 1.0274 - val_acc: 0.6805
Epoch 4/50
16832/50000 [=========>....................] - ETA: 5:03 - loss: 0.6905 - acc: 0.7585

50000/50000 [==============================] - 489s 10ms/step - loss: 0.6665 - acc: 0.7652 - val_loss: 0.9949 - val_acc: 0.7025
Epoch 5/50
15872/50000 [========>.....................] - ETA: 5:12 - loss: 0.5876 - acc: 0.7926

...

50000/50000 [==============================] - 489s 10ms/step - loss: 0.0718 - acc: 0.9739 - val_loss: 0.5549 - val_acc: 0.8846
Epoch 47/50
15232/50000 [========>.....................] - ETA: 5:17 - loss: 0.0627 - acc: 0.9770

50000/50000 [==============================] - 487s 10ms/step - loss: 0.0674 - acc: 0.9764 - val_loss: 0.5500 - val_acc: 0.8842
Epoch 48/50
15232/50000 [========>.....................] - ETA: 5:15 - loss: 0.0581 - acc: 0.9793

50000/50000 [==============================] - 485s 10ms/step - loss: 0.0629 - acc: 0.9778 - val_loss: 0.5704 - val_acc: 0.8741
Epoch 49/50
15232/50000 [========>.....................] - ETA: 5:14 - loss: 0.0575 - acc: 0.9798

50000/50000 [==============================] - 485s 10ms/step - loss: 0.0634 - acc: 0.9772 - val_loss: 0.5651 - val_acc: 0.8795
Epoch 50/50
15232/50000 [========>.....................] - ETA: 5:15 - loss: 0.0592 - acc: 0.9789

50000/50000 [==============================] - 486s 10ms/step - loss: 0.0641 - acc: 0.9774 - val_loss: 0.5590 - val_acc: 0.8861


In [32]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 38s 4ms/step
Test loss: 0.558982203412056
Test accuracy: 0.8861


In [33]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')